# Harvard Sociology Department Advising Network

## An Update

August 2017

In [1]:
# setup
import networkx as nx
import pandas as pd
import numpy as np

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
# read data
df = pd.read_excel('data/Advising_Data_JMM_Edits_August_2017.xls', sheetname=0)
df.head()

,StudentLast,StudentFirst,Committee1 (chair),Committee2,Committee3,Year,Committee4,Committee5,Unnamed: 8
0,Rubin,Julia,Skocpol,"Barnes, Louis","Bradach, Jeffrey",2001-2,"Ventresca, M",NaN,NaN
1,Gonzalez,Angela,Winship,Bobo,"Frank, David",2001-2,"Champagne, Duane",NaN,NaN
2,Gorman,Elizabeth,Marsden,Reskin,"Frank, David",2001-2,NaN,NaN,NaN
3,Frederick,Paula,Skocpol,Waters,"Frank, David",2001-2,NaN,NaN,NaN
4,Wasserman,Noam,Nohria,Baker,"Gompers, Paul",2001-2,Marsden,NaN,NaN


## 1. Create Faculty List

We will restrict the analysis of adivisng network to only current faculty. 

From email exchange with Jason:
* Include current (as well as emeritus) faculty and leave out recently-departed faculty -- excluding Filiz Garip, Tey Meadow, Matt Desmond, Nicholas Christakis, Chris Marquis, and Kathy Edin. But include Bruce Western, since, technically, he is on leave rather than resigned.

* 'Good' (whom we included in the first iteration of the network) is an affiliate. Best to omit her for this. 

* As for recently hired faculty, OK to omit faculty who have not served and are not serving on phd committees

In [3]:
df.columns

Index([       u'StudentLast',       u'StudentFirst', u'Committee1 (chair)',
               u'Committee2',         u'Committee3',               u'Year',
               u'Committee4',         u'Committee5',         u'Unnamed: 8'],
      dtype='object')

In [4]:
# put all committee columns into a list 
# then remove duplicate names
fac_list = df['Committee1 (chair)'].values.tolist() + df.Committee2.values.tolist() + \
            df.Committee3.values.tolist() + df.Committee4.values.tolist() + df.Committee5.values.tolist()
fac_list = list(set(fac_list))
print len(fac_list)

186


In [5]:
print fac_list

[nan, u'Zuckerman', u'Wilson', u'Eck, Diana', u'Reskin', u'Skocpol', u'Beckert, Sven', u'Nohria, Nitin', u'Schneider, Ben', u'Desmond', u'Margolis, Joshua', u'Kanter, Rosabeth', u'Sampson*', u'Ellwood, David', u'Ferguson, Ann', u'Lucchino, Lawrence', u'Podolny, Joel', u'Western', u'Weller, Robert', u'Lorsch, Jay', u'Wildeman, Christopher', u'McGinn', u'DiTomaso, Nancy', u'Marsden', u'Whyte*', u'Lieberson', u'Marquis', u'Ammerman', u'de Sousa-Briggs, Xavier', u'Christakis', u'Edin, Kathryn', u'Hackman, J. Richard', u'Simon, Patrick', u'Polzer, Jeffrey', u'Ammerman, Nancy', u'Coffin, Brent', u'Vogel', u'Higgonet, Patrice', u"O'Mahony, Siobhan", u'McDonald, Rory', u'Sorensen, Aage', u'Nanda, Ashish', u'Szelenyi, Ivan', u'Wuthnow, Robert', u'Edmondson', u'Jencks', u'Rainwater, Lee', u'Marsden*', u'Wein, Frederic', u'Schor, Juliet B.', u'Gulati, Ranjay', u'Winship*', u'Van Maanen, John', u'Battilana, Julie', u'Lewis, Mary', u'Stuart, Toby E.', u'Bobo', u'Good', u'Carpenter, Daniel', u'Anteb

In [6]:
# by comparing faculty list on soc website and unique names in the dataset above
own_fac_list = ['Wilson', 'Skocpol', 'Sampson', 'Western', 'Marsden', 'Whyte', 
                'Lieberson', 'Vogel', 'Jencks', 'Winship', 'Bobo', 'Brinton', 
                'Lei', 'Lamont', 'Dobbin', 'Bonikowski', 'Beckfield', 'Waters', 
                'Chang, Paul', 'Small', 'Killewald', 'Viterna', 'Patterson', 
                'Khurana, Rakesh', 'Pager']
print len(own_fac_list)

25


## 2. Clean Data

In [7]:
# get student full names

def get_student(row):
    """
    Combine student last name and first name.
    """
    
    if pd.isnull(row['StudentLast']):
        return None
    return row['StudentLast'].strip()+','+row['StudentFirst'].strip()

df['Student'] = df.apply(get_student, axis=1)
df.head()

,StudentLast,StudentFirst,Committee1 (chair),Committee2,Committee3,Year,Committee4,Committee5,Unnamed: 8,Student
0,Rubin,Julia,Skocpol,"Barnes, Louis","Bradach, Jeffrey",2001-2,"Ventresca, M",NaN,NaN,"Rubin,Julia"
1,Gonzalez,Angela,Winship,Bobo,"Frank, David",2001-2,"Champagne, Duane",NaN,NaN,"Gonzalez,Angela"
2,Gorman,Elizabeth,Marsden,Reskin,"Frank, David",2001-2,NaN,NaN,NaN,"Gorman,Elizabeth"
3,Frederick,Paula,Skocpol,Waters,"Frank, David",2001-2,NaN,NaN,NaN,"Frederick,Paula"
4,Wasserman,Noam,Nohria,Baker,"Gompers, Paul",2001-2,Marsden,NaN,NaN,"Wasserman,Noam"


In [8]:
# look for duplicates in students - compare row number and unique student number
print df.shape[0], len(df.Student.unique())

227 226


In [9]:
# remove duplicate
print df.shape
df = df.drop_duplicates('Student')
print df.shape

(227, 10)
(226, 10)


## 2. Create Bipartite Network

In [10]:
def find_faculty(fac_name, net):
    """
    Given fac_name and bipartite network graph net, 
    return the node if fac_name is already a node in net.
    Otherwise, return None.
    
    If multiple nodes are found associated with fac_name, raise Exception.
    """
    
    fac = [n for n,d in net.nodes(data=True) if n==fac_name]
    
    if len(fac) == 0:
        return None
    elif len(fac) == 1:
        return fac[0]
    else:
        raise Exception, 'ERROR: multiple nodes for '+fac_name
    

In [11]:
# create an empty network
b = nx.Graph()
    

# iterate through each row
for i, row in df.iterrows():
    
    # skip the empty row
    if pd.isnull(row['StudentLast']):
        continue
            
    # add student node
    # attributes: 'last name,first name', year, bipartite=0
    stud_name = row[0]+','+row[1]
    b.add_node(stud_name, year=row['Year'], bipartite=0)
        
    # add faculty nodes
    for col in ['Committee1 (chair)', 'Committee2', 'Committee3', 'Committee4', 'Committee5']:
        fac_name = row[col]
        
        if pd.isnull(fac_name): # not every student has 4 or 5 committee members
            break
        
        fac_name = fac_name.strip('*') # co-chairs are marked with asterisk, not used, remove     
            
        if fac_name in own_fac_list: # exclude faculty outside of soc dept
            # check if faculty is already added to the network
            fac = find_faculty(fac_name, b)
            
            if not fac:
                b.add_node(fac_name, bipartite=1)
                
            # add student-faculty tie
            b.add_edge(stud_name, fac_name)
            

## 3. Convert Bipartite to One-Mode Network

In [12]:
fac_nodes = [n for n,d in b.nodes(data=True) if d['bipartite']==1]
print len(fac_nodes) # SAME as the pre-specified faculty list, so every faculty has served/is serving on phd commmittee
print fac_nodes[:5] # print first five faculty names

25
[u'Wilson', u'Skocpol', u'Western', u'Marsden', u'Lieberson']


In [13]:
g = nx.bipartite.weighted_projected_graph(b, fac_nodes)
print g.edges(data=True)[:5] # print first five edges

[(u'Wilson', u'Bobo', {'weight': 6}), (u'Wilson', u'Beckfield', {'weight': 2}), (u'Wilson', u'Jencks', {'weight': 4}), (u'Wilson', u'Sampson', {'weight': 12}), (u'Wilson', u'Brinton', {'weight': 1})]


## 4. Export networkx Object to Gephi Object

In [14]:
# rename two faculty names before exporting
g = nx.relabel_nodes(g, {'Chang, Paul':'Chang', 'Khurana, Rakesh':'Khurana'})

In [15]:
nx.write_gexf(g, 'data/advising_network_2017.gexf')

## 5. Analyze Network

In [16]:
# find the tie with max weight
edge_data = g.edges(data=True)
weights = [e[2]['weight'] for e in edge_data]
weights.sort(reverse=True)
print weights

[16, 14, 12, 11, 11, 10, 9, 8, 8, 7, 7, 7, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [17]:
[e for e in edge_data if e[2]['weight']==16]

[(u'Patterson', u'Winship', {'weight': 16})]

In [18]:
[e for e in edge_data if e[2]['weight']==14]

[(u'Wilson', u'Waters', {'weight': 14})]

In [19]:
[e for e in edge_data if e[2]['weight']==12]

[(u'Wilson', u'Sampson', {'weight': 12})]

In [20]:
[e for e in edge_data if e[2]['weight']==11]

[(u'Wilson', u'Lamont', {'weight': 11}),
 (u'Wilson', u'Winship', {'weight': 11})]